# Query ConsDB

A basic notebook to learn about querying the ConsDb

Craig Lage - 17-Apr-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
import pandas as pd
import os
os.environ["no_proxy"] += ",.consdb"
from lsst.summit.utils import ConsDbClient
from lsst.summit.utils.utils import computeCcdExposureId

In [ ]:
client = ConsDbClient('http://consdb-pq.consdb:8080/consdb')
print(client.schema())  # list the instruments

In [ ]:
client.schema('lsstcam') # List tables for an instrument

In [ ]:
for table in client.schema('lsstcam'):
    tableName = table.split('.')[1]
    print(tableName)
    print()
    thisDict = client.schema('lsstcam', tableName)
    for key in thisDict.keys():
        print(key, thisDict[key])
        

In [ ]:
client.schema('lsstcam', 'visit1') # List values in a table

In [ ]:
client.schema('lsstcam', 'ccdvisit1_quicklook') # List values in a table

In [ ]:
client.schema('lsstcam', 'ccdexposure_quicklook') # List values in a table

In [ ]:
# Get all of the values for a particular table and visit
data = client.query('SELECT * from cdb_lsstcam.exposure')
for name in data.colnames:
    print(name, data[name].value[0])

In [ ]:
ccdvisit_id = computeCcdExposureId('lsstcam', 2024110900100, 95)
print(ccdvisit_id)


In [ ]:
ccdvisit_id = client.query('SELECT ccdvisit_id from cdb_lsstcam.ccdvisit1_quicklook')
for name in data.colnames:
    print(name, data[name].value[0])


In [ ]:
data['psf_sigma'][0]

In [ ]:
sig2fwhm = 2 * np.sqrt(2 * np.log(2))
pixel_scale = 0.2  # arcsec / pixel

for expId in range(2025112800100, 2025112800300):
    try:
        ccdvisit_id = computeCcdExposureId('lsstcam', expId, 94)
        #print(ccdvisit_id)
        data = client.query(f'SELECT ccdvisit1_quicklook.psf_sigma from cdb_lsstcam.ccdvisit1_quicklook where ccdvisit_id = {ccdvisit_id}')
        psf = data['psf_sigma'][0] * sig2fwhm * pixel_scale
        print(expId, psf)
    except:
        print(f"{expId} failed!")


In [ ]:

for expId in range(2025112800100, 2025112800300):
    try:
        ccdvisit_id = computeCcdExposureId('lsstcam', expId, 94)
        #print(ccdvisit_id)
        data = client.query(f'SELECT * from cdb_lsstcam.ccdvisit1_quicklook where ccdvisit_id = {ccdvisit_id}')
        print(expId, len(data.colnames))
    except:
        print(f"{expId} failed!")


In [ ]:
# I guess this just isn't populated yet?
data = client.query('SELECT ccdvisit_id from cdb_latiss.ccdvisit1_quicklook where ccdvisit_id > 1')
data

In [ ]:
# I guess this just isn't populated yet?
data = client.query('SELECT visit_id from cdb_latiss.visit1_quicklook where visit_id > 1')
data

In [ ]:
data = client.query('SELECT obs_start from cdb_lsstcomcamsim.exposure where day_obs = 20240621 and seq_num = 2')
data

In [ ]:
data = client.query('SELECT seq_num, obs_start, azimuth, zenith_distance from cdb_lsstcomcamsim.exposure where (seq_num > 0 and seq_num < 10 and day_obs = 20240621)')
data

In [ ]:
data = client.query('SELECT * from cdb_lsstcomcamsim.exposure where exposure_id = 7024062700764')
for name in data.colnames:
    print(name, data[name].value[0])


In [ ]:
def fetch(day_obs_min, day_obs_max):
    
    visits_query = f'''
    SELECT 
    visit1.visit_id,
    visit1.seq_num,
    visit1.band,
    visit1.day_obs,
    visit1.altitude,
    visit1.azimuth,
    visit1.sky_rotation,
    visit1_quicklook.aos_fwhm,
    visit1_quicklook.donut_blur_fwhm,
    visit1_quicklook.physical_rotator_angle
    FROM
    cdb_{instrument}.ccdvisit1_quicklook AS ccdvisit1_quicklook,
    cdb_{instrument}.ccdvisit1 AS ccdvisit1,
    cdb_{instrument}.visit1_quicklook AS visit1_quicklook,
    cdb_{instrument}.visit1 AS visit1 
    WHERE 
    ccdvisit1.ccdvisit_id = ccdvisit1_quicklook.ccdvisit_id
    AND ccdvisit1.visit_id = visit1.visit_id 
    AND visit1.visit_id = visit1_quicklook.visit_id
    AND ccdvisit1.detector NOT IN (168, 188, 123, 27, 0, 20, 65, 161)
    AND visit1.day_obs >= {day_obs_min} AND visit1.day_obs <= {day_obs_max}
    AND visit1.img_type = 'science';
    '''
    
    ccdvisits = client.query(visits_query).to_pandas()

    pixel_scale = 0.2
    sig2fwhm = 2 * np.sqrt(2 * np.log(2))
    
    return ccdvisits

In [ ]:
instrument = 'lsstcam'
ccdvisits = fetch(20251001, 20251129)
filtered_table = ccdvisits.groupby("visit_id").agg({col: 'first' if col == 'band' else 'mean' for col in ccdvisits.columns})

bands = ['u', 'g', 'r', 'i', 'z', 'y']
for band in bands:
    fwhm = filtered_table[filtered_table['band'] == band]
    print(band, len(fwhm))
    med = np.nanmedian(fwhm['aos_fwhm'].values)
    std = np.nanstd(fwhm['aos_fwhm'].values)
    plt.hist(fwhm['aos_fwhm'], bins=50, range=(0.2, 0.8), alpha=0.5, label=f"{band} - median={med:.2f} - std={std:.2f}")
    
plt.legend()

In [ ]:
groups['seq_num']

In [ ]:
t05 = groups['psf_fwhm'].quantile(0.05)
t95 = groups['psf_fwhm'].quantile(0.95)

In [ ]:
np.sqrt(t95**2 - t05**2)

In [ ]:
visits_summary['psf_fwhm_95_05']

In [ ]:
type(groups)

In [ ]:
groups.ngroups

In [ ]:
groups['psf_fwhm'].quantile(0.05)

In [ ]:
    
    AND (e.img_type = 'science' or e.img_type = 'acq' or e.img_type = 'engtest')


In [ ]:
def fetch(day_obs, seq_min, seq_max):
    
    visits_query = f'''
    SELECT 
    ccdvisit1_quicklook.psf_sigma,
    ccdvisit1.detector,
    visit1.visit_id,
    visit1.seq_num,
    visit1.day_obs,
    visit1.airmass
    FROM
    cdb_{instrument}.ccdvisit1_quicklook AS ccdvisit1_quicklook,
    cdb_{instrument}.ccdvisit1 AS ccdvisit1,
    cdb_{instrument}.visit1_quicklook AS visit1_quicklook,
    cdb_{instrument}.visit1 AS visit1 
    WHERE 
    ccdvisit1.ccdvisit_id = ccdvisit1_quicklook.ccdvisit_id
    AND ccdvisit1.visit_id = visit1.visit_id 
    AND visit1.visit_id = visit1_quicklook.visit_id
    AND ccdvisit1.detector NOT IN (168, 188, 123, 27, 0, 20, 65, 161)
    AND visit1.airmass > 0
    AND visit1.day_obs = {day_obs}
    AND (visit1.seq_num BETWEEN {seq_min} AND {seq_max})
    AND (visit1.img_type = 'science' or visit1.img_type = 'acq' or visit1.img_type = 'engtest')
    '''
    
    ccdvisits = client.query(visits_query).to_pandas()

    pixel_scale = 0.2
    sig2fwhm = 2 * np.sqrt(2 * np.log(2))
    ccdvisits["psf_fwhm"] = ccdvisits["psf_sigma"] * sig2fwhm * pixel_scale
    ccdvisits["psf_fwhm"] = pd.to_numeric(ccdvisits["psf_fwhm"], errors="coerce")

    return ccdvisits

In [ ]:
instrument = 'lsstcam'
ccdvisits = fetch(20251109, 0, 900)
len(ccdvisits)

In [ ]:
ccdvisits

In [ ]:
groups = ccdvisits.groupby('visit_id')

In [ ]:
t05 = groups['psf_fwhm'].quantile(0.05)
t95 = groups['psf_fwhm'].quantile(0.95)

In [ ]:
np.sqrt(t95**2 - t05**2)

In [ ]:
day_obs = 20251128
query = f'''
SELECT  
exposure.azimuth,
exposure.wind_dir,
exposure.wind_speed,
exposure_quicklook.mount_motion_image_degradation  
FROM 
cdb_latiss.exposure,
cdb_latiss.exposure_quicklook 
WHERE 
cdb_latiss.exposure.day_obs = {day_obs} 
AND
cdb_latiss.exposure_quicklook.day_obs = {day_obs} 
AND 
cdb_latiss.exposure.exposure_id = cdb_latiss.exposure_quicklook.exposure_id
'''
#print(query)
data = client.query(query).to_pandas()
len(data)

In [ ]:
day_obs = 20251128
query = f'''
SELECT  
exposure.azimuth,
exposure.wind_dir,
exposure.wind_speed,
exposure_quicklook.mount_motion_image_degradation  
FROM 
cdb_lsstcam.exposure,
cdb_lsstcam.exposure_quicklook 
WHERE 
cdb_lsstcam.exposure.day_obs = {day_obs} 
AND
cdb_lsstcam.exposure_quicklook.day_obs = {day_obs} 
AND 
cdb_lsstcam.exposure.exposure_id = cdb_lsstcam.exposure_quicklook.exposure_id
'''
#print(query)
data = client.query(query).to_pandas()
print(len(data))
for name in data.columns:
    print(name, data[name].values[0])


In [ ]:
day_obs = 20251129
seq_min = 0
seq_max = 900
instrument = 'lsstcam'

visits_query = f'''
SELECT 
ccdvisit1_quicklook.psf_sigma,
ccdvisit1.detector,
visit1.visit_id,
visit1.seq_num,
visit1.day_obs,
visit1.airmass
FROM
cdb_{instrument}.ccdvisit1_quicklook AS ccdvisit1_quicklook,
cdb_{instrument}.ccdvisit1 AS ccdvisit1,
cdb_{instrument}.visit1_quicklook AS visit1_quicklook,
cdb_{instrument}.visit1 AS visit1 
WHERE 
ccdvisit1.ccdvisit_id = ccdvisit1_quicklook.ccdvisit_id
AND ccdvisit1.visit_id = visit1.visit_id 
AND visit1.visit_id = visit1_quicklook.visit_id
AND ccdvisit1.detector NOT IN (168, 188, 123, 27, 0, 20, 65, 161)
AND visit1.day_obs = {day_obs}
AND (visit1.seq_num BETWEEN {seq_min} AND {seq_max})
'''

ccdvisits = client.query(visits_query).to_pandas()
print(len(ccdvisits))
print(max(ccdvisits['seq_num'].values))

In [ ]:
day_obs = 20251129
seq_min = 0
seq_max = 900
instrument = 'lsstcam'

visits_query = f'''
SELECT 
ccdvisit1.detector,
visit1.visit_id,
visit1.seq_num,
visit1.day_obs,
visit1.airmass
FROM
cdb_{instrument}.ccdvisit1 AS ccdvisit1,
cdb_{instrument}.visit1 AS visit1 
WHERE 
ccdvisit1.visit_id = visit1.visit_id 
AND ccdvisit1.detector NOT IN (168, 188, 123, 27, 0, 20, 65, 161)
AND visit1.day_obs = {day_obs}
AND (visit1.seq_num BETWEEN {seq_min} AND {seq_max})
'''

ccdvisits = client.query(visits_query).to_pandas()
print(len(ccdvisits))
print(max(ccdvisits['seq_num'].values))

In [ ]:
visits_query = f'''
SELECT 
ccdvisit1_quicklook.psf_sigma,
visit1.visit_id,
visit1.seq_num,
visit1.day_obs,
FROM
cdb_{instrument}.ccdvisit1_quicklook AS ccdvisit1_quicklook,
cdb_{instrument}.visit1_quicklook AS visit1_quicklook,
cdb_{instrument}.visit1 AS visit1 
WHERE 
ccdvisit1.ccdvisit_id = ccdvisit1_quicklook.ccdvisit_id
AND ccdvisit1.visit_id = visit1.visit_id 
AND visit1.visit_id = visit1_quicklook.visit_id
AND ccdvisit1.detector NOT IN (168, 188, 123, 27, 0, 20, 65, 161)
AND visit1.day_obs = {day_obs}
AND (visit1.seq_num BETWEEN {seq_min} AND {seq_max})
'''

ccdvisits = client.query(visits_query).to_pandas()
print(len(ccdvisits))
print(max(ccdvisits['seq_num'].values))

In [ ]:
data = client.query('SELECT * from cdb_lsstcam.exposure where (seq_num = 300 and day_obs = 20251118)')
data

In [ ]:
data = client.query('SELECT * from cdb_lsstcam.exposure_quicklook where (seq_num = 300 and day_obs = 20251118)')
data

In [ ]:
day_obs = 20251201
seq_min = 20
seq_max = 400
visits_query = f'''
SELECT 
visit1.visit_id,
visit1.guider_altitude_drift,
visit1.guider_azimuth_drift,
visit1.day_obs
FROM
cdb_lsstcam.visit1_quicklook AS visit1 
WHERE 
visit1.day_obs = {day_obs}
AND (visit1.seq_num BETWEEN {seq_min} AND {seq_max})
'''

visits = client.query(visits_query).to_pandas()
print(len(visits))


In [ ]:
for val in visits['guider_azimuth_drift'].values:
    print(val)
